# Overview

**Option Pricing using Black-Scholes** - ***by Felicity Bui***


*  Black-Scholes-Merton (BSM) - a differential equation.
*   BSM theory suggests that assets, like stocks or futures, exhibit a lognormal price distribution over time due to a random walk with steady drift and volatility.
*   BSM takes five input variables: the strike price of an option, the current stock price, the time to expiration, the risk-free rate, and the volatility to price option contracts.
*   The ultimate goal is to assist option sellers to determine rational prices for the options they offer. The model also anticipates that heavily traded assets follow a geometric Brownian motion with consistent drift and volatility. When applied to stock options, it factors in the stock's price variability, time value of money, strike price, and time until expiration.


# Black-Scholes model without live price call

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def black_scholes_call(S, K, r, t, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)

    call_price = S * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)

    return call_price

def calculate_option_prices(stock_price, strike_prices, r, t, sigma):
    option_prices = {}
    for strike_price in strike_prices:
        call_price = black_scholes_call(stock_price, strike_price, r, t, sigma)
        option_prices[strike_price] = call_price
        option_prices[strike_price] = round(call_price, 3)
    return option_prices

stock_prices = 100 #hypothesis
initial_strike = stock_prices - 10  # Start from 10 below previous close

# Check if the initial strike is below 25, between 25 and 200, or above 200
if initial_strike < 25:
    increment = 2.5
elif 25 <= initial_strike <= 200:
    increment = 5
else:
    increment = 10

strike_prices = []

# Generate strike prices
for i in range(5):
    strike_prices.append(initial_strike)
    initial_strike += increment

print(strike_prices)

# Other parameters (hypothetical values)
r = 0.0458   # Risk-free interest rate
t = 0.5    # Time to maturity (in years)
sigma = 0.2  # Volatility

# Calculate option prices
option_prices = calculate_option_prices(stock_prices, strike_prices, r, t, sigma)

# Print example option prices for each strike price
for strike_price, prices in option_prices.items():
    print(f"Strike Price: {strike_price}")
    print("Option Prices:", prices)
    print()

[90, 95, 100, 105, 110]
Strike Price: 90
Option Prices: 13.351

Strike Price: 95
Option Prices: 9.741

Strike Price: 100
Option Prices: 6.778

Strike Price: 105
Option Prices: 4.495

Strike Price: 110
Option Prices: 2.843



# Black-Scholes model with live price call from yahoo.com

get html table structure

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://finance.yahoo.com/quote/AMZN?.tsrc=fin-srch"

# Fetch the HTML content of the webpage
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table element(s)
tables = soup.find_all('table')

# Print the structure of table for testing
#for i, table in enumerate(tables, start=1):
#    print(f"Table {i} Structure:")
#    print(table)
#    print()


**define functions**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from scipy.stats import norm

def black_scholes_call(S, K, r, t, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)

    call_price = S * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)

    return call_price

def calculate_option_prices(stock_price, strike_prices, r, t, sigma):
    option_prices = {}
    for strike_price in strike_prices:
        call_price = black_scholes_call(stock_price, strike_price, r, t, sigma)
        option_prices[strike_price] = call_price
        option_prices[strike_price] = round(call_price, 3)
    return option_prices

def get_previous_close(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}"
    response = requests.get(url)
    #print(response.content)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the HTML element containing the previous close price
    previous_close_element = soup.find("span", text="Previous Close")
    # Check if the element was found
    if previous_close_element:
        # If the element was found, extract the next sibling element which contains the value
        previous_close = previous_close_element.find_next_sibling("span")
        # Convert the text to a float and remove commas
        previous_close = previous_close.text.strip()
        previous_close = float(previous_close.replace(',', ''))
        return previous_close
    else:
        # If the element was not found, return None or raise an error
        return None


# Get previous close price of stock
ticker_symbol = "GOOG"
previous_close_price = get_previous_close(ticker_symbol)

# Print the previous close price
print(ticker_symbol + " Previous Close Price:", previous_close_price)
#print(type(previous_close_price))


GOOG Previous Close Price: 170.9


<ipython-input-5-65c2797acdb3>:30: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  previous_close_element = soup.find("span", text="Previous Close")


Generate hypothetical values - strike prices and other parameters

In [ ]:
initial_strike = previous_close_price - 10  # Start from 10 below previous close

# Check if the initial strike is below 25, between 25 and 200, or above 200
if initial_strike < 25:
    increment = 2.5
elif 25 <= initial_strike <= 200:
    increment = 5
else:
    increment = 10

strike_prices = []

# Generate strike prices
for i in range(5):
    strike_prices.append(initial_strike)
    initial_strike += increment

print(strike_prices)

# Other parameters
r = 0.0458   # Risk-free interest rate
t = 0.5    # Time to maturity (in years)
sigma = 0.2  # Volatility

[160.9, 165.9, 170.9, 175.9, 180.9]


Call function

In [ ]:
# Calculate option prices
stock_price = previous_close_price  # Assign the single stock price
option_prices = calculate_option_prices(stock_price, strike_prices, r, t, sigma)

# Print example option prices for each strike price
for strike_price, prices in option_prices.items():
    print(f"Strike Price: {strike_price}")
    print("Option Prices:", prices)
    print()

Strike Price: 160.9
Option Prices: 17.623

Strike Price: 165.9
Option Prices: 14.406

Strike Price: 170.9
Option Prices: 11.584

Strike Price: 175.9
Option Prices: 9.162

Strike Price: 180.9
Option Prices: 7.129

